In [1]:
# Importing necessary libraries
import numpy as np
import scipy as scp
import time

## Matrix Multiplication

Adding an example with matrix multiplication to give you an example of what you can do to implement LU

In [2]:
# Function for manual matrix multiplication
def manual_matrix_multiplication(A, B):
    m, n = A.shape
    n_, p = B.shape
    
    if n != n_:
        print("Incompatible matrices. Can't perform multiplication.")
        return
    
    # Create the result matrix and fill it with zeros
    result = np.zeros((m, p))

    # Performing matrix multiplication
    for i in range(m): # iterate over rows
        for j in range(p): # iterate over columns
            for k in range(n): # summing over all k
                result[i, j] += A[i, k] * B[k, j]
    
    return result


In [3]:
# Initialize random matrices A and B
m = 100
n = 200
p = 300
A = np.random.rand(m,n)
B = np.random.rand(n,p)

In [4]:
# Time and run numpy matrix multiplication
start_time = time.time()
numpy_result = np.dot(A, B)
end_time = time.time()
print(f"NumPy matrix multiplication took {end_time - start_time} seconds.")

# Time and run manual matrix multiplication
start_time = time.time()
manual_result = manual_matrix_multiplication(A, B)
end_time = time.time()
print(f"Manual matrix multiplication took {end_time - start_time} seconds.")

print("Are both results equal?", np.allclose(manual_result, numpy_result))

NumPy matrix multiplication took 0.001001119613647461 seconds.
Manual matrix multiplication took 2.837029218673706 seconds.
Are both results equal? True


In [5]:
# Function for manual matrix multiplication
def manual_matrix_multiplication(A, B):
    m, n = A.shape
    n_, p = B.shape
    
    if n != n_:
        print("Incompatible matrices. Can't perform multiplication.")
        return
    
    # Create the result matrix and fill it with zeros
    result = np.zeros((m, p))

    # Performing matrix multiplication
    for i in range(m): # iterate over rows
        for j in range(p): # iterate over columns
            for k in range(n): # summing over all k
                result[i, j] += A[i, k] * B[k, j]
    
    return result

## Homework 1 - LU decomposition

Code for calculating the decomposition PA = LU. A is the input matrix P is a permutation matrix, L is triangular inferior and U is triangular superior.
Write your implementation in the function below

In [6]:
def your_LU_decomposition(A):
    n = A.shape[0]
    P = np.eye(n)
    L = np.eye(n)
    U = np.copy(A)            
    
    # LU with pivoting implementation
    for i in range(n):
        # Pivoting
        max_index = np.argmax(abs(U[i:, i])) + i
        if max_index != i:
            # Swap the elements of P and U
            P[[i, max_index], :] = P[[max_index, i], :]
            U[[i, max_index], :] = U[[max_index, i], :]
            # Swap the elements of L if i >= 1
            if i >= 1:
                L[[i, max_index], :i] = L[[max_index, i], :i]
        
        # Elimination
        for j in range(i+1, n):
            L[j, i] = U[j, i] / U[i, i]
            U[j, i:] -= L[j, i] * U[i, i:]
    
    return P, L, U

In [7]:
# Functions for checking the algorithms
def check_is_permutation(P):
    n = P.shape[0]
    if P.shape[1] != n:
        return False	

    if np.all((P == 0) | (P == 1)) and \
            np.all(np.sum(P, axis=0) == 1) and \
                np.all(np.sum(P, axis=1) == 1):
        return True
    else:
        return False
    

def test_LU(A, P, L, U):
    
    if not check_is_permutation(P):
        return False
    
    if not np.allclose(L, np.tril(L)):
        return False

    if not np.allclose(U, np.triu(U)):
        return False
    
    return np.allclose(P @ A, L @ U)  

After your implementation, run the cell below:

In [8]:
A = np.random.rand(100, 100)

# Time and run scipy LU
start_time = time.time()
P, L, U = scp.linalg.lu(A)
# Scipy gives us PLU decomposition, so we need to transpose P
P = P.T
end_time = time.time()
print(f"Scipy's LU decomposition took {end_time - start_time} seconds.")
if test_LU(A, P, L, U):
    print("And it is correct!")
else:
    print("And it is incorrect!")

# Time and run LU decomposition
start_time = time.time()
P, L, U = your_LU_decomposition(A)
end_time = time.time()
print(f"Your LU decomposition took {end_time - start_time} seconds.")
if test_LU(A, P, L, U):
    print("And it is correct!")
else:
    print("And it is incorrect!")

Scipy's LU decomposition took 0.15945124626159668 seconds.
And it is correct!
Your LU decomposition took 0.01623249053955078 seconds.
And it is correct!


## Homework 2 - Echelon form

Code for calculating the echelon form of A. A is the input matrix and it should return (P, L, U), where P is a permutation matrix, L is triangular inferior and U is the echelon form. It should be PA = LU. 
Write your implementation in the function below

In [9]:
def your_echelon_form(A, tol=1e-10):
    # Because of numerical error, one often does not get exact 0 on elimination 
    # You should declare that there is no pivot in a column if all elements 
    # in that column are smaller than tol
    
    m, n = A.shape
    P = np.eye(m)
    L = np.eye(m)
    U = np.copy(A)
    
    # Implement your Echelon form algorithm here            
    
    return P, L, U

In [10]:
# Functions for checking the algorithms
def check_is_permutation(P):
    n = P.shape[0]
    if P.shape[1] != n:
        return False	

    if np.all((P == 0) | (P == 1)) and \
            np.all(np.sum(P, axis=0) == 1) and \
                np.all(np.sum(P, axis=1) == 1):
        return True
    else:
        return False

def test_echelon_form(U, tol=1e-10):
    # Test that matrix U is echelon form
    m, n = U.shape
    pivot_column = -1
    for i in range(m):
        # Find pivot (first element in row bigger than tol in absolute value)
        pivot_column, old_pivot_column = np.nonzero(abs(U[i, :]) > tol)[0][0], pivot_column

        if pivot_column.size == 0:
            pivot_column = n + 1
        elif pivot_column <= old_pivot_column:
            return False
        
    return True

def test_echelon_decomposition(A, P, L, U, tol=1e-10):
    
    if not check_is_permutation(P):
        return False
    
    if not np.allclose(L, np.tril(L)):
        return False

    if not test_echelon_form(U, tol=1e-10):
        return False
    
    return np.allclose(P @ A, L @ U)

In [16]:
A = np.random.rand(100, 100)

P, L, U = scp.linalg.lu(np.random.rand(100, 100))
P, L, U = scp.linalg.lu(np.random.rand(100, 100))

# Time and run scipy LU
start_time = time.time()
P, L, U = scp.linalg.lu(A)
# Scipy gives us PLU decomposition, so we need to transpose P
P = P.T
end_time = time.time()
print(f"Scipy's LU decomposition took {end_time - start_time} seconds.")
if test_echelon_decomposition(A, P, L, U):
    print("And it is correct!")
else:
    print("And it is incorrect!")

# Time and run LU decomposition
start_time = time.time()
P, L, U = your_LU_decomposition(A)
end_time = time.time()
print(f"Your LU decomposition took {end_time - start_time} seconds.")
if test_echelon_decomposition(A, P, L, U):
    print("And it is correct!")
else:
    print("And it is incorrect!")

Scipy's LU decomposition took 0.004428863525390625 seconds.
And it is correct!
Your LU decomposition took 0.015688657760620117 seconds.
And it is correct!
